In [1]:
import numpy as np
import pandas as pd
import sys

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
train['Age'] = train['Age'].fillna(train['Age'].median())

In [5]:
train['Embarked'] = train['Embarked'].fillna('S')

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [7]:
X_train = train[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']]

In [8]:
X_train['Sex'][X_train['Sex'] == 'male'] = 0
X_train['Sex'][X_train['Sex'] == 'female'] = 1

X_train['Embarked'][X_train['Embarked'] == 'S'] = 0
X_train['Embarked'][X_train['Embarked'] == 'C'] = 1
X_train['Embarked'][X_train['Embarked'] == 'Q'] = 2

/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/envs/pytorch/lib/python3.6/site-packages/pandas/core/generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user

In [9]:
Y_train = train['Survived']

In [10]:
class LogisticRegression(object):
    
    def __init__(self, eta=0.0001, epoch=1000, randam_state=1, \
                alpha=0, shuffle=False):
        self.eta = eta
        self.epoch = epoch
        self.randam_state = randam_state
        self.alpha = alpha
        self.shuffle = shuffle
        
    def fit(self, X, y):
        print(X.shape)
        
        X = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
        rgen = np.random.RandomState(self.randam_state)
        self._w = rgen.normal(loc=0.0, scale=0.01, size=X.shape[1])
        self._cost = []
        lr_w = np.zeros(X.shape[1])
        
        for i in range(self.epoch):
            w_grad = np.zeros(X.shape[1])
            
            if self.shuffle:
                X, y = self._shuffle(X, y)
                
            for xi, target in zip(X,y):
                cost = []
                output = self.sigmoid(self.net_input(xi))
                error = -np.sum(target*np.log(output) + (1-target)*np.log(1- output))
                w_grad = w_grad - xi.dot(target - output)
                cost.append(error)
            
            lr_w = lr_w + w_grad**2
            
#             self._w = self._w - self.eta / lr_w * w_grad
            self._w = self._w - self.eta * w_grad
            
            if i%200 == 0:
                print('Epoch: ' + str(i) + ' , cost: ' + str(cost))
            
            self._cost.append(abs(np.average(cost)))
        
        return self
    
    def sigmoid(self, z):
        z = np.array(z,dtype=np.float32)
        res = 1 / (1.0 + np.exp(-z))
        return np.clip(res, 1e-10, 1-1e-10)
    
    def net_input(self, X):
        return np.dot(X, self._w)
    
    def predict(self, X):
        X = np.concatenate((np.zeros((X.shape[0], 1)), X), axis=1)
        return self.sigmoid(self.net_input(X))
    
    def _shuffle(self, X, y):
        r = np.random.permutation(len(y))
        return X[r], y[r]

def sign(a):
    output = []
    for i in a:
        if i<0.5:
            output.append(0)
        else:
            output.append(1)
    return output

In [11]:
X_train['Pclass'] = (X_train['Pclass'] - X_train['Pclass'].median()) / X_train['Pclass'].std()

/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
X_train['Age'] = (X_train['Age'] - X_train['Age'].median()) / X_train['Age'].std()
X_train['Fare'] = (X_train['Fare'] - X_train['Fare'].median()) / X_train['Fare'].std()

/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
Lm = LogisticRegression()
Lm.fit(X_train, Y_train)

(891, 5)
Epoch: 0 , cost: [0.6761681642938339]
Epoch: 200 , cost: [0.2651657074429555]
Epoch: 400 , cost: [0.20592221793138196]
Epoch: 600 , cost: [0.17746664122622952]
Epoch: 800 , cost: [0.16114254332052955]


In [14]:
X_test = test[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']]

X_test['Sex'][X_test['Sex'] == 'male'] = 0
X_test['Sex'][X_test['Sex'] == 'female'] = 1

X_test['Embarked'][X_test['Embarked'] == 'S'] = 0
X_test['Embarked'][X_test['Embarked'] == 'C'] = 1
X_test['Embarked'][X_test['Embarked'] == 'Q'] = 2

/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/pytorch/lib/python3.6/site-packages/pandas/core/generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

In [15]:
X_test['Age'] = X_test['Age'].fillna(X_test['Age'].median())

/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].median())

/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
X_test.isnull().sum()

Pclass      0
Sex         0
Age         0
Fare        0
Embarked    0
dtype: int64

In [18]:
X_test['Pclass'] = (X_test['Pclass'] - X_test['Pclass'].median()) / X_test['Pclass'].std()
X_test['Age'] = (X_test['Age'] - X_test['Age'].median()) / X_test['Age'].std()
X_test['Fare'] = (X_test['Fare'] - X_test['Fare'].median()) / X_test['Fare'].std()

/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [19]:
pred_y = sign(Lm.predict(X_test))

In [21]:
Y_test = pd.read_csv('data/gender_submission.csv')

In [24]:
np.sum(pred_y == Y_test['Survived'])/len(Y_test)

0.4904306220095694